## Bike Index Seattle - Data Prep

### Data cleaning for Seattle streets data

##### Objective: Recreate the study by Allen-Munley et al. (2004) for Seattle using WSDOT crash data.


#### Part 2.

I will take Seattle street data from [SDOT](https://data-seattlecitygis.opendata.arcgis.com/datasets/seattle-streets), which contain the following [attributes](https://www.seattle.gov/Documents/Departments/SDOT/GIS/Seattle_Streets_OD.pdf).

The coordinates from the WSDOT crashes do not correspond to the street network coordinates, so I will first find the nearest street segment from the geodata, and then take the projected coordinate on the street vector as the new crash coordinate, in order to merge the two tables to get street attributes.

[Reference for snapping to nearest street](https://medium.com/@brendan_ward/how-to-leverage-geopandas-for-faster-snapping-of-points-to-lines-6113c94e59aa)



In [43]:
import requests
import json
import pandas as pd
import numpy as np
import geopandas as gpd
import shapely
from shapely.geometry import Point, LineString
import os
import pathlib
import matplotlib.pyplot as plt

In [44]:
%matplotlib inline

In [138]:
#url = 'https://data.seattle.gov/resource/38vd-gytv.json'
#Didn't use

In [143]:
!git push origin main

To https://github.com/emi90/bike-index.git
   60719dc..f843795  main -> main


#### Step 1: Load data

- Load street data from SDOT geoJSON file
- Load crash data from previous step
- Convert street geoJSON and crash data to geoDataFrame

In [146]:
# Load street data from SDOT

gisurl = 'https://opendata.arcgis.com/datasets/383027d103f042499693da22d72d10e3_0.geojson'

r = requests.get(gisurl)

streets = r.json()

In [148]:
# Load crash data from previous step

crashes = pd.read_csv('data/bike_crash.csv')

In [150]:
# Convert street geoJSON and crash data .csv to geoDataFrame


# Convert lat/long to geometry
geom = [Point(xy) for xy in zip(crashes.LONGITUDE, crashes.LATITUDE)]

gdf_crashes = gpd.GeoDataFrame(crashes.drop(['LATITUDE','LONGITUDE'], axis = 1),
                               geometry = geom)

gdf = gpd.GeoDataFrame.from_features(streets)

#### Step 2: Find closest street segment for every crash point coordinate

- Given offset parameter, find the acceptable bounds for each crash coordinate
- Find street segments within the bounds for each crash coordinate
- Find closest street segment for each given crash coordinate
- 'Snap' crash coordinate to the street segment vector by projection

In [152]:
# Create index for street geoDF
gdf.sindex

# Select offset parameter- approx 10m
offset = 10**-4 

# Create bound box for each coordinate given offset parameter
bbox = gdf_crashes.bounds + [-offset, -offset, offset, offset]

In [153]:
# Find street segments within bounds for each crash coordinate

hits = bbox.apply(lambda row: list(gdf.sindex.intersection(row)), axis=1)

In [110]:
plt.rcParams["figure.figsize"] = (20,20)

In [112]:
offset = 10**-4 ## approx 10m

bbox = gdf_crashes.bounds + [-offset, -offset, offset, offset]

In [114]:
tmp = pd.DataFrame({
    # index of points table
    "pt_idx": np.repeat(hits.index, hits.apply(len)),
    # ordinal position of line - access via iloc later
    "line_i": np.concatenate(hits.values)
})

In [115]:
# Join back to the lines on line_i; we use reset_index() to 
# give us the ordinal position of each line

tmp1 = tmp.merge(gdf.reset_index(drop=False).rename(columns={'index':'line_i'}), on="line_i")
# Join back to the original points to get their geometry
# rename the point geometry as "point"

#### Step 4: Network-specific variables

The Allen-Munley et al. study uses the following variables for their study.  

![Explanatory Variables](variables.png)  

The injuries will be mapped using the following index: 
- `PDO - NO INJURY CRASHES` : 1
- `POSSIBLE INJURIES` : 2
- `EVIDENT INJURIES` : 3
- `SERIOUS INJURIES` : 3
- `FATALITIES` : 4

In [116]:
crashes_tmp = gdf_crashes.reset_index(drop=False).rename(columns={'geometry':'point', 'index':'pt_idx'})

In [117]:
crashes_tmp.columns

Index(['pt_idx', 'REPORT NUMBER', 'BLOCK NUMBER', 'PRIMARY TRAFFICWAY',
       'INTERSECTING TRAFFICWAY', 'DATETIME', 'severity', 'is_dry', 'is_light',
       'is_clear', 'is_hit_run', 'TOTAL VEHICLES',
       'TOTAL PEDESTRIANS INVOLVED', 'TOTAL BICYCLISTS INVOLVED', 'point'],
      dtype='object')

In [118]:
tmp2 = tmp1.merge(crashes_tmp, on="pt_idx")
# Convert back to a GeoDataFrame, so we can do spatial ops

tmp3 = gpd.GeoDataFrame(tmp2, geometry="geometry", crs=gdf_crashes.crs)

In [119]:
tmp3["snap_dist"] = tmp3.geometry.distance(gpd.GeoSeries(tmp3.point))

In [120]:
max(tmp3.snap_dist)

0.015461724901658874

In [121]:
tolerance = offset # keep at same distance, approx 10m

# Discard any lines that are greater than tolerance from points
tmp4 = tmp3.loc[tmp3.snap_dist <= tolerance]
# Sort on ascending snap distance, so that closest goes to top
tmp4 = tmp4.sort_values(by=["snap_dist"])

In [123]:
# group by the index of the points and take the first, which is the
# closest line 
closest = tmp4.groupby("pt_idx").first()
# construct a GeoDataFrame of the closest lines
closest = gpd.GeoDataFrame(closest, geometry="geometry")

In [126]:
#randomly check 10 from closest street vs csv file

check_ix = np.random.randint(len(closest)-1, size=10)

closest.iloc[check_ix][['UNITDESC','PRIMARY TRAFFICWAY','INTERSECTING TRAFFICWAY']]

,UNITDESC,PRIMARY TRAFFICWAY,INTERSECTING TRAFFICWAY
pt_idx,,,
972,NE 53RD ST BETWEEN RAVENNA AVE NE AND 24TH AVE NE,RAVENNA PL NE,None
472,DEXTER AVE N BETWEEN VALLEY ST AND ALOHA ST,DEXTER AVE N,ALOHA ST
262,63RD AVE SW BETWEEN SW ADMIRAL WAY AND SW HIND...,63RD AVE SW,None
1141,STONE WAY N BETWEEN N 36TH ST AND N 38TH ST,STONE WAY N,None
763,N 109TH ST BETWEEN FREMONT AVE N AND NORTH PAR...,N 109TH ST,None
89,1ST AVE BETWEEN PINE ST AND STEWART ST,1ST AVE,None
1326,E LAKE WASHINGTON BLVD BETWEEN MONTLAKE BLVD E...,520LX00094,None
705,NW 39TH ST BETWEEN LEARY WAY NW AND 3RD AVE NW,LEARY WAY NW,NW 39TH ST
82,18TH AVE BETWEEN E PINE ST AND E MADISON ST,18TH AVE,None


In [127]:
# Position of nearest point from start of the line
pos = closest.geometry.project(gpd.GeoSeries(closest.point))

# Get new point location geometry
new_pts = closest.geometry.interpolate(pos)

In [137]:
# Create a new GeoDataFrame from the columns from the closest line and new point geometries (which will be called "geometries")

crashes_merged = gpd.GeoDataFrame(closest.drop(columns = ['geometry']),geometry=new_pts)